<a href="https://colab.research.google.com/github/Sam-Gartenstein/GenAI-Engineering-Workshop/blob/main/Part%202%3A%20Using%20LLM%20Agents%20for%20Essay%20Generation%20and%20Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 2

You have completed [Part 1: Guide to OpenAI in Google Colab](https://github.com/Sam-Gartenstein/GenAI-Engineering-Workshop/blob/main/Part%201%3A%20Guide%20to%20OpenAI%20in%20Google%20Colab.ipynb). Now, let’s apply those skills to a real-world use case.  

In this section, you’ll learn about **[role-based collaboration](https://www.ibm.com/think/topics/multi-agent-collaboration)**, a framework where multiple LLMs work together by taking on distinct roles with specific purposes. Each agent performs its own task and passes the result to the next, creating a seamless workflow powered by **prompt chaining**.  

We will use generative AI to:

- **Create** an 11th-grade English essay (Student Agent)  
- **Design** a rubric to evaluate the essay (Rubric Agent)  
- **Grade** the essay using the rubric (Grading Agent)  

By the end of this section, you’ll understand how to design and connect multiple role-based agents to simulate a realistic classroom workflow — from writing to evaluation to grading.


First, let's repeat the steps we followed in the previous notebook, which was importing the necessary libraries and loading our key.

**Note**: Once you have made a key in colab, it will automatically save, meaning you can use it in other notebooks. When running line two for the first time, you will get a pop up window asking you to grant access.

In [1]:
import openai
from openai import OpenAI
from google.colab import userdata
import os

In [2]:
# Pull your saved secret into an environment variable
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# Test if the key is available (without printing it)
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY is not set. Add it via Colab Secrets (🔑) and try again.")
else:
    print("Key loaded?", True)

Key loaded? True


## Generating Essays

Let's start by generating two essays on the same topic — **Shakespeare’s *Othello***. We’ll create one **A-level** and one **C-level** essay to compare how writing quality changes when the same Student Agent follows different performance prompts.

The function we use, `generate_essay`, produces a single essay at a time.  
In addition to the main inputs (`system_role`, `prompt`, `model`, and `temperature`), it also takes two key arguments:

- **`topic`** — The essay question or theme.  
  This defines *what* the student is writing about. For example:  
  *“Discuss how jealousy shapes Othello’s decisions and relationships.”*

- **`grade_level`** — A label that identifies the educational level or target audience for the essay.  
  Here, it helps maintain context, such as *“11th grade,”* so the model writes with an appropriate voice and complexity. If not familiar with the US education system, **please read** this [article](https://usahello.org/education/children/grade-levels/) explaining the US grade systems.

By generating both essays on the same topic, we can clearly see how **prompt design** shapes differences in reasoning depth, structure, and tone.

Additionally, we will use the markdown format function!


In [3]:
def generate_essay(
    system_role: str,
    prompt: str,
    topic: str,
    grade_level: str,
    model: str = "gpt-4o-mini",
    temperature: float = 0.2,
) -> str:
    """
    Generate a synthetic essay using the OpenAI Responses API.
    Keep it simple: pass output requirements inside `prompt`.
    """
    user_content = (
        f"{prompt}\n\n"
        f"ESSAY TOPIC / QUESTION:\n{topic}\n\n"
        f"TARGET GRADE LEVEL: {grade_level}\n"
    )

    resp = client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": system_role},
            {"role": "user", "content": user_content},
        ],
        temperature=temperature,
    )
    return resp.output_text

In [4]:
from IPython.display import display, Markdown  # Tools for displaying formatted text in Jupyter Notebooks

def to_markdown(text):
    # Convert the provided text to Markdown format for better display in Jupyter Notebooks
    return Markdown(text)


## Essay Generation

As shown in [Part 1](https://github.com/Sam-Gartenstein/GenAI-Engineering-Workshop/blob/main/Part%201%3A%20Guide%20to%20OpenAI%20in%20Google%20Colab.ipynb) of this course series, we will be using system/user role prompting. Our goal is to generate essays of varying performance levels, with one consistent Student Agent representing the writer’s identity, and the user prompt controlling the essay’s quality.

The Student Agent defines the persona — a realistic high school student writing for an English class — while the prompt specifies the assignment requirements and the expected performance level. By adjusting the temperature and prompt details, we can simulate essays ranging from polished and insightful (A-level) to basic and error-prone (C-level).

This setup allows us to maintain a consistent writing voice while producing diverse samples for grading, evaluation, or fine-tuning models that assess writing quality.

### Understanding Letter Grades

In many U.S. schools and universities, essays are graded on a **letter scale** from **A** to **F**. For more information, please read [Grading System in USA](https://www.canamgroup.com/blog/grading-system-in-usa).

First, as always, let's creates a client object.

In [5]:
client = OpenAI()

### Student Agent

The Student Agent represents an 11th-grade student writing an essay for an English class. This role defines the writer’s voice — thoughtful but not overly advanced — ensuring consistency across essays of different quality levels.


In [6]:
student_agent = (
    "You are an 11th-grade high school student writing an essay for your English class. "
    "Write in a realistic student voice — thoughtful but not overly advanced. "
    "Use natural language, as if written by a real student rather than an adult academic."
)

### A-Level Prompt

The **A-Level Prompt** instructs the model to produce an outstanding literary analysis essay that meets the highest standards of academic writing. An *A-level* essay demonstrates deep understanding, originality, and strong analytical reasoning. It presents a clear, arguable thesis — placed at the end of the introduction — and supports that argument with carefully selected textual evidence and insightful interpretation.  

Writing at this level should be polished, precise, and logically structured. The essay must avoid casual tone, first-person language, and rhetorical questions, maintaining a formal academic voice throughout. Each paragraph should begin with a clear topic sentence and connect cohesively to the thesis, creating a unified, persuasive argument.  

In our prompt, the model is **explicitly guided** to follow these conventions. It is asked to use short, purposeful quotations from the play and analyze *how* they support the central argument, emphasizing analysis over summary. The target length is approximately **450–600 words**, with citations formatted as *(act.scene.line)* (e.g., *3.3.167*). These clear and specific instructions ensure that the model’s output resembles a well-crafted, high-performing literary analysis essay appropriate for an advanced high school student.


In [7]:
A_level_prompt = (
    "Write an A-level literary analysis essay that follows the detailed guidelines below.\n\n"
    "What 'A-level' means: excellent or outstanding work — demonstrates a deep and "
    "original understanding of the text, presents a clear and arguable thesis, and supports claims with precise "
    "evidence and insightful analysis. Writing should be organized, polished, and stylistically sophisticated.\n\n"
    "OUTPUT REQUIREMENTS:\n"
    "- Include a clear, arguable thesis at the END of the introduction.\n"
    "- Use concise, well-chosen quotations from the play and explain HOW they support your argument "
    "(analysis > summary).\n"
    "- Ensure each paragraph has a strong topic sentence and smooth transitions; every idea should advance the thesis.\n"
    "- Maintain a precise, academic tone throughout; avoid first person, rhetorical questions, or casual phrasing.\n"
    "- Aim for ~450–600 words.\n"
    "- Cite quotations using act.scene.line format (e.g., 3.3.167).\n"
)


In [8]:
topic = "How does Iago exploit Othello’s insecurities to drive the tragedy in *Othello*?"
grade_level = "11th grade"

A_level_essay = generate_essay(
    system_role=student_agent,
    prompt=A_level_prompt,
    topic=topic,
    grade_level=grade_level,
    model="gpt-4o-mini",
    temperature=0.3,
)


In [9]:
to_markdown(A_level_essay)

**Title: The Manipulation of Insecurity in Shakespeare's *Othello***

In William Shakespeare's *Othello*, the tragic downfall of the titular character is intricately woven through the manipulative actions of Iago, who exploits Othello’s insecurities to orchestrate his demise. Othello, a Moor and a general in the Venetian army, is portrayed as a noble and capable leader; however, his status as an outsider in Venetian society renders him vulnerable to doubt and jealousy. Iago, understanding this vulnerability, skillfully manipulates Othello’s insecurities regarding his race, his marriage to Desdemona, and his self-worth. Through Iago’s cunning machinations, Shakespeare illustrates how unchecked insecurities can lead to devastating consequences, ultimately driving the tragedy of Othello.

Iago’s exploitation of Othello’s racial insecurities is a central theme in the play. Othello’s position as a black man in a predominantly white society subjects him to prejudice and doubt. Iago recognizes this vulnerability and uses it to his advantage. For instance, he refers to Othello as “the Moor” (1.1.8), a term that strips Othello of his individuality and reduces him to his race. This dehumanizing language serves to reinforce Othello’s outsider status, making him more susceptible to Iago’s manipulations. Furthermore, Iago’s insinuations about Desdemona’s fidelity play on Othello’s fears of inadequacy. When Iago suggests that Desdemona may be unfaithful, he exploits Othello’s internalized racism, implying that a woman of Desdemona’s beauty could not genuinely love someone of Othello’s background. This manipulation is evident when Iago states, “Blessed fig's-end! The wine she drinks is made of grapes” (2.3.319), suggesting that Desdemona’s love is superficial and conditional. By planting these seeds of doubt, Iago deepens Othello’s insecurities, leading him to question his worthiness as a husband.

Moreover, Iago capitalizes on Othello’s insecurities regarding his marriage to Desdemona. Othello’s love for Desdemona is profound, yet his lack of confidence in her loyalty becomes a weapon in Iago’s hands. Iago’s calculated insinuations create a rift between the couple, as seen when he tells Othello, “O, beware, my lord, of jealousy! It is the green-eyed monster which doth mock the meat it feeds on” (3.3.165-167). This metaphor not only highlights the destructive nature of jealousy but also serves to instill fear in Othello. By framing jealousy as an inevitable consequence of love, Iago manipulates Othello into believing that his insecurities are justified. This manipulation culminates in Othello’s tragic transformation from a dignified leader to a man consumed by jealousy and rage, ultimately leading to the tragic climax of the play.

Iago’s manipulation extends to Othello’s self-worth, as he systematically dismantles Othello’s confidence. Throughout the play, Othello grapples with feelings of inadequacy, particularly in his role as a husband and a leader. Iago’s relentless insinuations about Desdemona’s fidelity exacerbate these feelings, leading Othello to question his identity. When Othello declares, “I am not what I am” (1.1.65), he reflects the internal conflict that Iago has instigated. This statement encapsulates Othello’s struggle with his self-perception, as Iago’s deceit forces him to confront the insecurities he has long tried to suppress. The tragic irony lies in the fact that Othello’s noble qualities—his bravery, intelligence, and love for Desdemona—become overshadowed by the insecurities that Iago has so expertly exploited.

In conclusion, Iago’s exploitation of Othello’s insecurities regarding race, marriage, and self-worth serves as a catalyst for the tragedy that unfolds in *Othello*. Through his manipulative tactics, Iago not only reveals the vulnerabilities of Othello but also highlights the destructive power of jealousy and doubt. Shakespeare’s portrayal of this tragic descent underscores the profound impact that insecurities can have on an individual’s psyche, ultimately leading to their downfall. Othello’s tragic fate serves as a poignant reminder of the dangers of allowing insecurities to dictate one’s actions and perceptions, a theme that resonates with audiences across time.

-----

### C-Level Prompt

The **C-Level Prompt** instructs the model to produce an average or barely passing essay. A *C-level* essay demonstrates basic comprehension of the topic but lacks depth, organization, and stylistic control. It may have a weak or missing thesis, minimal or poorly integrated evidence, repetitive ideas, and noticeable grammar or usage mistakes.  

Writing at this level should sound more casual and unpolished — including filler phrases such as “I think” or “in my opinion” and avoiding strong transitions like “however” or “therefore.” The essay should be short, approximately four paragraphs (200–300 words), and contain a few minor grammatical errors that sound natural for a struggling student writer.  

Like before, we provide **explicit instructions** in the prompt to guide the model’s behavior. The detailed output requirements ensure the essay intentionally reflects weaker academic performance while still maintaining a realistic student voice.

In [11]:
C_level_prompt = (
    "Write an essay responding to the question below.\n\n"
    "IMPORTANT: Produce a deliberately poor-quality **C-level** essay for teaching purposes.\n"
    "What 'C-level' means: average or barely passing work — shows some understanding "
    "but lacks depth or polish; weak thesis or none, little/no evidence, poor organization, repetition/vagueness, "
    "and noticeable grammar/style mistakes.\n\n"
    "OUTPUT REQUIREMENTS:\n"
    "- Return ONLY the essay body (no headings or bullets).\n"
    "- Do NOT provide a clear thesis; keep the main claim vague.\n"
    "- Provide minimal or poorly integrated evidence (generic statements are fine; avoid direct quotations).\n"
    "- Organization should be weak (some repetition or loosely connected ideas is acceptable).\n"
    "- Keep it short: 4 paragraphs (~200–300 words).\n"
    "- Include a few minor grammar/usage mistakes naturally (e.g., comma splices, agreement issues).\n"
    "- Casual, somewhat imprecise tone is acceptable.\n"
    "- Use a few filler phrases like 'I think' or 'in my opinion,' and avoid strong transitions like 'however' or 'therefore.'"
)


C_level_essay = generate_essay(
    system_role=student_agent,
    prompt=C_level_prompt,
    topic=topic,
    grade_level=grade_level,
    model="gpt-4o-mini",
    temperature=0.3,
)

In [12]:
to_markdown(C_level_essay)

In the play *Othello*, Iago is a character who uses Othello’s insecurities to create a lot of problems. Othello is a strong general, but he has some doubts about himself, especially because he is not from Venice and is black. I think Iago sees this and takes advantage of it. He makes Othello question his wife Desdemona’s loyalty, which is a big deal for Othello. This shows how Iago is manipulative and knows how to push Othello’s buttons.

One way Iago exploits Othello’s insecurities is by planting seeds of doubt. He talks about Desdemona being with Cassio, which makes Othello feel insecure about his marriage. Othello starts to believe that Desdemona might not love him as much as he loves her. This is really sad because Othello is a good guy, but Iago twists his mind. I think this is a big part of why the tragedy happens. Othello becomes jealous and angry, and it leads to bad choices.

Also, Iago uses Othello’s race against him. Othello feels like he has to prove himself to everyone, and Iago knows this. He makes comments that play into Othello’s fears about being an outsider. This makes Othello more vulnerable to Iago’s lies. It’s like Iago is playing a game, and Othello is just a pawn. I think it’s really unfair how Iago treats him, but that’s what makes the story tragic.

In conclusion, Iago’s manipulation of Othello’s insecurities is a key part of the tragedy in the play. Othello’s doubts about himself and his relationship with Desdemona lead to a lot of misunderstandings. Iago is clever in how he uses these insecurities, and it shows how dangerous manipulation can be. Overall, the play is a good example of how someone can exploit another person’s weaknesses for their own gain, which is really sad.

----

## Rubric Generation

Great! We’ve now generated two essays on the same topic but with different performance levels. The next step is to create a **rubric** that can be used to evaluate them.

This rubric function works almost exactly like the `generate_essay` function. It still takes in the same core arguments — `system_role`, `prompt`, `topic`, and `grade_level`. The only difference is in how the user message is structured.

Inside the `user_content`, instead of including `"ESSAY TOPIC / QUESTION:\n{topic}\n\n"` we replace it with

`"RUBRIC TOPIC / CONTEXT:\n{topic}\n\n"`.

This small change tells the model that it should generate an **evaluation framework** rather than an essay. The output will describe the criteria, expectations, and performance distinctions (for example, what separates A-level from C-level work) for that specific topic or assignment.


In [13]:
def generate_essay_rubric(
    system_role: str,
    prompt: str,          # include OUTPUT REQUIREMENTS here (categories, A–F bands, format)
    topic: str,           # e.g., the essay question or assignment name
    grade_level: str,     # e.g., "11th grade"
    model: str = "gpt-4o-mini",
    temperature: float = 0.1,
) -> str:
    """
    Generate a grading rubric using the OpenAI Responses API.
    Keep it simple: pass rubric requirements inside `prompt`.
    """
    user_content = (
        f"{prompt}\n\n"
        f"RUBRIC TOPIC / CONTEXT:\n{topic}\n\n"
        f"TARGET GRADE LEVEL: {grade_level}\n"
    )

    resp = client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": system_role},
            {"role": "user", "content": user_content},
        ],
        temperature=temperature,
    )
    return resp.output_text

### First Teacher Agent and Prompt

Now, we can create our first **Teacher Agent**, which will be responsible for designing grading rubrics. Since this agent focuses on rubric generation rather than essay writing, we’ll call it the `rubric_agent`.

The `rubric_agent` represents a **high school English teacher** who creates clear, student-facing grading rubrics. This agent interprets essay prompts and outlines performance expectations across categories such as Thesis, Evidence, Analysis, Organization, and Mechanics.

To generate a rubric, we pass in a short instructional prompt that tells the model to create a structured grading guide. The output should include 4–5 evaluation categories, each with descriptions for performance levels **A through F**, written concisely in Markdown format. The temperature is kept low (0.1) to ensure consistent and structured output.


In [14]:
rubric_agent = (
    "You are a high school English teacher. "
    "Design clear, student-facing grading rubrics."
)

prompt = (
    "Create a rubric for the essay prompt below.\n\n"
    "OUTPUT REQUIREMENTS:\n"
    "- Use 4–5 categories (e.g., Thesis, Evidence, Analysis, Organization, Mechanics).\n"
    "- For each category, define performance levels: A, B, C, D, F.\n"
    "- Keep descriptions concise and specific.\n"
    "- Format in Markdown with headings and bullet points."
)

In [ ]:
topic = "How does Iago exploit Othello’s insecurities to drive the tragedy in *Othello*?"

othello_rubric = generate_essay_rubric(
    system_role=rubric_agent,
    prompt=prompt,
    topic=topic,
    grade_level="11th grade",
    model="gpt-4o-mini",
    temperature=0.1,
)

In [15]:
to_markdown(othello_rubric)

# Essay Grading Rubric: Iago's Exploitation of Othello's Insecurities

## Categories

### 1. Thesis
- **A**: Clear, insightful thesis that directly addresses the prompt and presents a unique perspective.
- **B**: Clear thesis that addresses the prompt but lacks depth or originality.
- **C**: Thesis is present but vague or only partially addresses the prompt.
- **D**: Weak thesis that does not clearly address the prompt.
- **F**: No thesis present.

### 2. Evidence
- **A**: Uses multiple relevant and specific textual examples to support claims effectively.
- **B**: Uses relevant textual examples, but may lack depth or specificity in some areas.
- **C**: Provides some textual evidence, but it may be insufficient or not clearly connected to the argument.
- **D**: Minimal evidence provided; examples are weak or irrelevant.
- **F**: No evidence provided.

### 3. Analysis
- **A**: Insightful and thorough analysis of how Iago exploits Othello’s insecurities; connects evidence to thesis effectively.
- **B**: Good analysis that connects evidence to the thesis, but may lack depth in some areas.
- **C**: Basic analysis that connects some evidence to the thesis, but lacks depth or clarity.
- **D**: Weak analysis; connections between evidence and thesis are unclear or missing.
- **F**: No analysis present.

### 4. Organization
- **A**: Well-organized essay with clear, logical flow; effective use of paragraphs and transitions.
- **B**: Generally organized with minor issues in flow or paragraph structure.
- **C**: Some organization present, but may be confusing or lack clear transitions.
- **D**: Poorly organized; difficult to follow the argument.
- **F**: No clear organization; chaotic structure.

### 5. Mechanics
- **A**: Virtually no errors in grammar, punctuation, or spelling; polished writing style.
- **B**: Few minor errors that do not interfere with understanding; generally clear writing.
- **C**: Noticeable errors that may distract from the content but do not impede understanding.
- **D**: Frequent errors that interfere with understanding; unclear writing.
- **F**: Severe errors that make the essay unreadable.

---

### Total Score
- **A (90-100)**: Excellent understanding and execution of the prompt.
- **B (80-89)**: Good understanding with minor issues.
- **C (70-79)**: Satisfactory understanding but needs improvement.
- **D (60-69)**: Limited understanding; significant issues present.
- **F (below 60)**: Unsatisfactory; does not meet basic requirements.

## Essay Grading

Now, let's get to the heart of this task, using the LLM to actually grade the essay! In addition to our normal arguments, this function will now take in the essay text and our rubric!

In [16]:
def grade_essay_with_rubric_md(
    system_role: str,
    prompt: str,          # tell it HOW to grade + the desired Markdown format
    rubric_md: str,       # the rubric text (Markdown or plain text)
    essay_text: str,      # the student essay
    grade_level: str,     # e.g., "11th grade"
    model: str = "gpt-4o-mini",
    temperature: float = 0.2, # low temperature for consistency
) -> str:
    """
    Grade an essay using the provided rubric and return a concise Markdown report.
    The model should mirror whatever categories exist in rubric_md.
    """
    user_content = (
        f"{prompt}\n\n"
        f"RUBRIC (Markdown allowed):\n{rubric_md}\n\n"
        f"ESSAY TO GRADE:\n{essay_text}\n\n"
        f"TARGET GRADE LEVEL: {grade_level}\n"
    )

    resp = client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": system_role},
            {"role": "user", "content": user_content},
        ],
        temperature=temperature,
    )
    return resp.output_text


### Second Teacher Agent and Prompt

Let's define the grading agent! The **Grading Agent** is a high school English teacher whose job is to evaluate an essay **using the provided rubric**. This agent prioritizes fairness, consistency, and clarity. Feedback is professional and concise.

The grading prompt instructs the model to return a **Markdown report** with two sections:

- **Overall Grade**: includes a performance level (A–F), a numeric score (0–100), and a brief 2–3 sentence summary.
- **Category Breakdown**: follows the rubric’s categories **exactly and in order**, giving each a level (A–F), a numeric score (0–100), and a 1–2 sentence justification tied to the rubric criteria.

Rules enforced by the prompt:
- Mirror the rubric categories verbatim and keep the same order.
- Base judgments on evidence from the essay and the rubric (no invented criteria).
- Keep explanations brief, specific, and directly connected to the rubric.

In [17]:
grading_agent = (
    "You are a high school English teacher and careful grader. "
    "Use the provided rubric to evaluate the essay fairly and consistently. "
    "Your feedback should be clear, concise, and professional."
)

prompt = (
    "Evaluate the essay using the provided rubric.\n\n"
    "OUTPUT FORMAT (Markdown):\n"
    "## Overall Grade\n"
    "- **Level:** A|B|C|D|F\n"
    "- **Score:** 0–100\n"
    "- **Summary (2–3 sentences):** …\n\n"
    "## Category Breakdown\n"
    "For each category from the rubric (in the same order), include:\n"
    "- **Level:** A|B|C|D|F\n"
    "- **Score:** 0–100\n"
    "- **Justification (1–2 sentences):** specific, tied to the rubric\n\n"
    "Rules:\n"
    "- Mirror the rubric categories exactly and in order.\n"
    "- Be concise and evidence-based.\n"
    "- Do not invent categories that are not in the rubric."
)

#### Grading A Level Essay

In [18]:
A_essay_eval = grade_essay_with_rubric_md(
    system_role=grading_agent,
    prompt=prompt,
    rubric_md=othello_rubric,
    essay_text=A_level_essay,
    grade_level="11th grade",
)
to_markdown(A_essay_eval)

## Overall Grade
- **Level:** A
- **Score:** 95
- **Summary:** This essay presents a clear and insightful thesis that effectively addresses the prompt regarding Iago's exploitation of Othello's insecurities. The analysis is thorough, supported by relevant textual evidence, and the organization is logical and coherent.

## Category Breakdown

### 1. Thesis
- **Level:** A
- **Score:** 95
- **Justification:** The thesis is clear and insightful, directly addressing the prompt while presenting a unique perspective on Iago's manipulation of Othello's insecurities.

### 2. Evidence
- **Level:** A
- **Score:** 90
- **Justification:** The essay uses multiple relevant textual examples effectively to support claims, demonstrating a strong connection to the thesis, though a few examples could benefit from deeper exploration.

### 3. Analysis
- **Level:** A
- **Score:** 95
- **Justification:** The analysis is insightful and thorough, effectively connecting evidence to the thesis and exploring the implications of Iago's manipulation on Othello's psyche.

### 4. Organization
- **Level:** A
- **Score:** 95
- **Justification:** The essay is well-organized with a clear, logical flow and effective use of paragraphs and transitions, making it easy to follow the argument.

### 5. Mechanics
- **Level:** A
- **Score:** 95
- **Justification:** The writing is polished with virtually no errors in grammar, punctuation, or spelling, contributing to a clear and engaging style.

-----

#### Analysis

The grading output shows that the LLM effectively followed the rubric and grading prompt, producing a clear and well-structured evaluation. It begins with an overall grade that includes the level, score, and summary, followed by a detailed category breakdown that mirrors the rubric. Each section—Thesis, Evidence, Analysis, Organization, and Mechanics—uses consistent formatting and concise justifications tied directly to the rubric.

The model assigns mostly A-level scores, with slight variation such as a 90 for Evidence to reflect minor areas for improvement. Its feedback is professional and balanced, combining praise with targeted suggestions like expanding certain examples. Notably, the model graded the essay as an **A**, aligning perfectly with the essay’s intended quality level. This confirms that the LLM correctly understood both the essay-generation and grading instructions, demonstrating reliable rubric-based evaluation and teacher-like judgment.


#### Grading C Level Essay

In [19]:
C_essay_eval = grade_essay_with_rubric_md(
    system_role=grading_agent,
    prompt=prompt,
    rubric_md=othello_rubric,
    essay_text=C_level_essay,
    grade_level="11th grade",
)
to_markdown(C_essay_eval)

## Overall Grade
- **Level:** C
- **Score:** 75
- **Summary:** The essay presents a satisfactory understanding of Iago's exploitation of Othello's insecurities but lacks depth and clarity in its thesis and analysis. While it provides some relevant examples, the connections between evidence and argument could be more developed.

## Category Breakdown

### 1. Thesis
- **Level:** C
- **Score:** 70
- **Justification:** The thesis is present but vague, stating that Iago exploits Othello's insecurities without offering a unique perspective or deeper insight into the implications of this manipulation.

### 2. Evidence
- **Level:** C
- **Score:** 70
- **Justification:** The essay provides some relevant textual examples, such as Iago's manipulation regarding Desdemona and Othello's race, but lacks depth and specificity in how these examples support the overall argument.

### 3. Analysis
- **Level:** C
- **Score:** 70
- **Justification:** The analysis connects some evidence to the thesis but lacks depth and clarity. The exploration of Iago's manipulation could be more thorough and insightful to strengthen the argument.

### 4. Organization
- **Level:** B
- **Score:** 80
- **Justification:** The essay is generally organized with a clear structure, but transitions between ideas could be improved for a smoother flow of argument.

### 5. Mechanics
- **Level:** B
- **Score:** 80
- **Justification:** The writing contains few minor errors that do not interfere with understanding, but there are moments where clarity could be enhanced for a more polished presentation.

#### Analysis

Once again, the LLM effectively followed the rubric and grading prompt to produce a structured and balanced evaluation. The model graded the essay as a **C**, matching the intended performance level from the prompt. The feedback highlights key weaknesses—such as a vague thesis, limited analytical depth, and underdeveloped connections between evidence and argument—while still acknowledging strengths in organization and mechanics. This shows that the model can accurately differentiate between levels of writing quality and apply the rubric criteria fairly and consistently. Overall, the output reflects reliable, evidence-based grading aligned with the expectations of a high school English teacher.


## Forward

You have now seen how we can use LLMs to create agents that write, evaluate, and grade essays through **role-based collaboration**. Each agent — the Student, Teacher, and Grading Agent — performs a distinct function, and together they form a connected workflow powered by **prompt chaining**. This process allows outputs from one model to serve as inputs for another, mirroring how humans collaborate in structured tasks like writing and assessment.

Now, the rest of the notebook is up to you — it’s your turn to experiment! Try extending what you’ve learned in creative ways. For example, you can:

- Generate **D- or F-level** essays to simulate weaker student writing  
- **Adjust the existing prompts** to modify tone, structure, or grading criteria  
- **Create additional agents** such as a Peer Reviewer, Rubric Validator, or Format Checker  
- Modify the **grading rubric** to target different skills or subject areas  

Exploring these variations will help you deepen your understanding of how **prompt design**, **role specialization**, and **collaboration** shape the quality, reasoning, and reliability of LLM-generated outputs.
